In [1]:
using StaticArrays, DifferentialEquations, DynamicalSystems, CairoMakie

In [19]:
function TM(u, p, t)

    U(y) = U0 + ΔU0 / ( 1 + exp( -50*(y-ythr) ) )
    σ(x) = 1 / ( 1 + exp( -20*(x-xthr) ) )

    E, x, y = u
    α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0 = p

    du1 = (-E + α * log( 1 + exp( (J * U(y) * x * E + I0  ) / (α) ) ) ) / τ
    du2 = (1-x)/τD - U(y)*x*E
    du3 = (-y)/τy+β*σ(x)

    return SVector(du1, du2, du3)
end

TM (generic function with 1 method)

In [20]:
function count_floats(a, b)
           count = 0
           if a > b
               a, b = b, a
           end
           while a < b
               a = nextfloat(a)
               count += 1
           end
           return count
end

count_floats (generic function with 1 method)

In [21]:
t = 1000.0
tt = 500.0
tstep = 0.001
trange = range(0.0, t, step = tstep);
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [22]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58; const U0 = 0.3;  const ΔU0 = 0.305;

In [23]:
I0 = -1.4
p =  [α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [8.39353, 0.73803, 0.429055]
ds = ContinuousDynamicalSystem(TM, u0, p)

3-dimensional continuous dynamical system
 state:       [8.39353, 0.73803, 0.429055]
 rule f:      TM
 in-place?    false
 jacobian:    ForwardDiff
 parameters:  [1.58, 0.013, 0.08, 3.3, 3.07, 0.75, 0.4, 0.3, 0.305, 0.3, -1.4]

In [24]:
count_floats(Float32(-1.40), Float32(-1.87036))

3945666

In [25]:
I0range = range( -1.4, -1.87036, length = 2001 )

idx_control_parameter = 11

idx_fix = 2; fixed_value = 0.75
surface = (idx_fix, fixed_value)
setting_root = (xrtol = 1e-13, atol = 1e-13);

In [26]:
I0range

-1.4:-0.00023518:-1.87036

In [ ]:
idx_save = 1
output = produce_orbitdiagram(ds, surface, idx_save,
                        idx_control_parameter, I0range;
                        tfinal = t, Ttr = tt,
                        printparams = true,
                        diffeq = integ_set, rootkw = setting_root
                        );

parameter = -1.4
parameter = -1.40023518
parameter = -1.40047036
parameter = -1.40070554
parameter = -1.40094072
parameter = -1.4011759
parameter = -1.40141108
parameter = -1.40164626
parameter = -1.40188144
parameter = -1.40211662
parameter = -1.4023518
parameter = -1.40258698
parameter = -1.40282216
parameter = -1.40305734
parameter = -1.40329252
parameter = -1.4035277
parameter = -1.40376288
parameter = -1.40399806
parameter = -1.40423324
parameter = -1.40446842
parameter = -1.4047036
parameter = -1.40493878
parameter = -1.40517396
parameter = -1.40540914
parameter = -1.40564432
parameter = -1.4058795
parameter = -1.40611468
parameter = -1.40634986
parameter = -1.40658504
parameter = -1.40682022
parameter = -1.4070554
parameter = -1.40729058

In [ ]:
f = Figure(resolution = (1200, 400))
axis = Axis(f[1,1]; xlabel = L"I_0", ylabel = L"E", xlabelsize = 30, ylabelsize = 30,
            xticklabelsize = 25, yticklabelsize = 25, xgridvisible = false, ygridvisible = false, backgroundcolor = :black)
for (j, p) in enumerate(I0range)
    scatter!(axis, fill(p, length(output[j])), output[j]; color = ("deeppink", 0.5), markersize = 1.5)
end
f